In [1]:
#import dependencies 
import pandas as pd
import numpy as np
import datetime
import math

In [2]:
#read the csv file
df = pd.read_csv("Raw data/07-12data.csv")
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,634,2013-07-01 00:00:00,2013-07-01 00:10:34,164,E 47 St & 2 Ave,40.753231,-73.970325,504,1 Ave & E 15 St,40.732219,-73.981656,16950,Customer,\N,0
1,1547,2013-07-01 00:00:02,2013-07-01 00:25:49,388,W 26 St & 10 Ave,40.749718,-74.002950,459,W 20 St & 11 Ave,40.746745,-74.007756,19816,Customer,\N,0
2,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,40.730287,-73.990765,237,E 11 St & 2 Ave,40.730473,-73.986724,14548,Subscriber,1980,2
3,1580,2013-07-01 00:01:06,2013-07-01 00:27:26,531,Forsyth St & Broome St,40.718939,-73.992663,499,Broadway & W 60 St,40.769155,-73.981918,16063,Customer,\N,0
4,757,2013-07-01 00:01:10,2013-07-01 00:13:47,382,University Pl & E 14 St,40.734927,-73.992005,410,Suffolk St & Stanton St,40.720664,-73.985180,19213,Subscriber,1986,1


In [3]:
#check the number of rows 
#check dtypes of the dataframe
print(df.shape)
print(df.dtypes)

(5037185, 15)
tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                  object
gender                       int64
dtype: object


In [4]:
#convert dtype trip duration column from object to integer 
#replace anything that is not a number with 0
df["tripduration"] =pd.to_numeric(df["tripduration"], errors = 'coerce').fillna(0).astype(int)

In [5]:
#drop rows that were less than 90 seconds or more than 24 hours
#riders should take more than 2 minutes from each station 
#riders shouldn't take more than 24hrs from each station 
df= df[~((df["tripduration"]<90) |(df["tripduration"]>86400))]

In [6]:
#drop rows that started and ended in the same station less than a 5 min
#station A to station B shouldn't be less than 5 mins - if so probably issues occured within the bike or rider 
df= df[~((df["tripduration"]<=300) & (df["start station id"] == df["end station id"]))]

In [7]:
#convert dtype of column starttime, set errors='coerce' for invalid values
#convert starttime coulumn to the format year-month-day hour-min-sec
df['starttime'] = pd.to_datetime(df['starttime'],errors='coerce', format='%Y-%m-%d %H:%M:%S')

In [8]:
#convert dtype of column stoptime, set errors='coerce' for invalid values
#convert stopttime coulumn to the format year-month-day hour-min-sec
df['stoptime'] = pd.to_datetime(df['stoptime'],errors='coerce', format='%Y-%m-%d %H:%M:%S')

In [ ]:
#convert dtype for column start station id where valid
df["start station id"] =pd.to_numeric(df["start station id"], errors = 'coerce').fillna(0).astype(int)
#drop station id 0
df=df[~(df["start station id"] ==0)]

In [9]:
#convert dtype for column end station id where valid
df["end station id"] =pd.to_numeric(df["end station id"], errors = 'coerce').fillna(0).astype(int)
#drop station id 0
df=df[~(df["end station id"] ==0)]

In [10]:
#convert dtype for all of the given lat and long
df[["start station latitude", 
    "start station longitude", 
    "end station latitude", 
    "end station longitude"]] = df[["start station latitude", 
                                    "start station longitude",
                                    "end station latitude",
                                    "end station longitude"]].apply(pd.to_numeric)

In [11]:
#convert birth year from object to int
df["birth year"] = pd.to_numeric(df["birth year"], errors = 'coerce').fillna(0).astype(int)

In [12]:
#drop birth year 0 (if any) or year 1930 and less
df= df[~((df["birth year"] == 0) | (df["birth year"] <= 1930))]

In [13]:
#convert bikeid from object to int
df["bikeid"] = pd.to_numeric(df["bikeid"]).astype(int)

In [15]:
#convert dtype gender
df.gender = df.gender.astype(str)

In [16]:
df.dtypes

tripduration                        int32
starttime                  datetime64[ns]
stoptime                   datetime64[ns]
start station id                    int64
start station name                 object
start station latitude            float64
start station longitude           float64
end station id                      int32
end station name                   object
end station latitude              float64
end station longitude             float64
bikeid                              int32
usertype                           object
birth year                          int32
gender                             object
dtype: object

In [18]:
#replace numbers with genders under gender column
df['gender'].replace({"0": "Unknown", "1": "Male", "2": "Female"}, inplace=True)

In [19]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
2,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,40.730287,-73.990765,237,E 11 St & 2 Ave,40.730473,-73.986724,14548,Subscriber,1980,Female
4,757,2013-07-01 00:01:10,2013-07-01 00:13:47,382,University Pl & E 14 St,40.734927,-73.992005,410,Suffolk St & Stanton St,40.720664,-73.985180,19213,Subscriber,1986,Male
5,861,2013-07-01 00:01:23,2013-07-01 00:15:44,511,E 14 St & Avenue B,40.729387,-73.977724,454,E 51 St & 1 Ave,40.754557,-73.965930,16223,Subscriber,1988,Male
7,288,2013-07-01 00:02:16,2013-07-01 00:07:04,224,Spruce St & Nassau St,40.711464,-74.005524,376,John St & William St,40.708621,-74.007222,16062,Subscriber,1985,Female
8,766,2013-07-01 00:02:16,2013-07-01 00:15:02,432,E 7 St & Avenue A,40.726218,-73.983799,336,Sullivan St & Washington Sq,40.730477,-73.999061,17963,Subscriber,1980,Female


In [20]:
#drop any nan values
df.dropna()
df.reset_index(drop=True)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,40.730287,-73.990765,237,E 11 St & 2 Ave,40.730473,-73.986724,14548,Subscriber,1980,Female
1,757,2013-07-01 00:01:10,2013-07-01 00:13:47,382,University Pl & E 14 St,40.734927,-73.992005,410,Suffolk St & Stanton St,40.720664,-73.985180,19213,Subscriber,1986,Male
2,861,2013-07-01 00:01:23,2013-07-01 00:15:44,511,E 14 St & Avenue B,40.729387,-73.977724,454,E 51 St & 1 Ave,40.754557,-73.965930,16223,Subscriber,1988,Male
3,288,2013-07-01 00:02:16,2013-07-01 00:07:04,224,Spruce St & Nassau St,40.711464,-74.005524,376,John St & William St,40.708621,-74.007222,16062,Subscriber,1985,Female
4,766,2013-07-01 00:02:16,2013-07-01 00:15:02,432,E 7 St & Avenue A,40.726218,-73.983799,336,Sullivan St & Washington Sq,40.730477,-73.999061,17963,Subscriber,1980,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4332735,1014,2013-12-31 23:55:56,2014-01-01 00:12:50,350,Clinton St & Grand St,40.715595,-73.987030,532,S 5 Pl & S 4 St,40.710451,-73.960876,16336,Subscriber,1973,Male
4332736,682,2013-12-31 23:57:04,2014-01-01 00:08:26,538,W 49 St & 5 Ave,40.757952,-73.977876,2006,Central Park S & 6 Ave,40.765909,-73.976342,18352,Subscriber,1986,Male
4332737,1520,2013-12-31 23:57:44,2014-01-01 00:23:04,281,Grand Army Plaza & Central Park S,40.764397,-73.973715,411,E 6 St & Avenue D,40.722281,-73.976687,15000,Subscriber,1985,Male
4332738,1166,2013-12-31 23:58:14,2014-01-01 00:17:40,492,W 33 St & 7 Ave,40.750200,-73.990931,253,W 13 St & 5 Ave,40.735439,-73.994539,21022,Subscriber,1955,Male


In [21]:
#calculate age 
df["age"] = 2022 - df["birth year"]

In [22]:
#drop birth year column
del df["birth year"]

In [23]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,gender,age
2,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,40.730287,-73.990765,237,E 11 St & 2 Ave,40.730473,-73.986724,14548,Subscriber,Female,42
4,757,2013-07-01 00:01:10,2013-07-01 00:13:47,382,University Pl & E 14 St,40.734927,-73.992005,410,Suffolk St & Stanton St,40.720664,-73.985180,19213,Subscriber,Male,36
5,861,2013-07-01 00:01:23,2013-07-01 00:15:44,511,E 14 St & Avenue B,40.729387,-73.977724,454,E 51 St & 1 Ave,40.754557,-73.965930,16223,Subscriber,Male,34
7,288,2013-07-01 00:02:16,2013-07-01 00:07:04,224,Spruce St & Nassau St,40.711464,-74.005524,376,John St & William St,40.708621,-74.007222,16062,Subscriber,Female,37
8,766,2013-07-01 00:02:16,2013-07-01 00:15:02,432,E 7 St & Avenue A,40.726218,-73.983799,336,Sullivan St & Washington Sq,40.730477,-73.999061,17963,Subscriber,Female,42


In [25]:
#save the clean data
df.to_csv("Raw data/CleanData.csv")